# Detecção e Classificação de Exsudatos na Retina<br>Utilizando Redes Neurais Convolucionais

<h2>Introdução</h2>

O presente trabalho apresenta diferentes maneiras de aplicar técnicas de Processamento Digital de Imagens para detectar e classificar a presença ou asência de exsudatos na retina.

#### Com exsudatos
![alt text](database/ex/0.png "Esxudato")

#### Sem exsudatos
![alt text](database/normal/0.png "Normal")

<h2>Extração de características e classificação</h2>

<p> <font size='3'>Primeiramente, será aplicado o filtro <b>prewitt</b> com a intenção de deixa a região de interesse mais evidente. </font></p>

In [ ]:
from skimage.io import imread
from skimage.exposure import histogram
from skimage.color import rgb2gray
import glob as g
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import skew, kurtosis, entropy
%matplotlib inline

labels, labels_name = [], ['normal', 'ex']

normal = g.glob('./data_set/normal/*.png')
ex = g.glob('./data_set/ex/*.png')

images = []

for path in normal:
    im = imread(path, as_grey=True)
    images.append(im)
    labels.append(0)

for path in ex:
    im = imread(path, as_grey=True)
    images.append(im)
    labels.append(1)

## Librarys

In [ ]:
import glob 
import random

import numpy as np
from skimage.io import imread_collection
import tensorflow as tf

## Define Layers

In [ ]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1], padding='SAME')

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

## Building Architecture of the Network

In [ ]:
x = tf.placeholder(tf.float32, [None, 1, 8464])

y_ = tf.placeholder(tf.float32, [None, 2])

x_image = tf.reshape(x, [-1,92,92,1])

W_conv1 = weight_variable([3, 3, 1, 32])

b_conv1 = bias_variable([32])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)

h_pool1 = max_pool_2x2(h_conv1)

W_conv2 = weight_variable([3, 3, 32, 64])

b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)

h_pool2 = max_pool_2x2(h_conv2)

W_fc1 = weight_variable([33856, 1024])

b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 33856])

h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float32)

h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W_fc2 = weight_variable([1024, 2])

b_fc2 = bias_variable([2])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

## Training

In [ ]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits= y_conv, labels = y_))

train_step = tf.train.AdamOptimizer(1e-5).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

## Spliting data

In [ ]:
def read_images(path):
    classes = glob.glob(path+'*')
    im_files = []
    size_classes = []
    for i in classes:
        name_images_per_class = glob.glob(i+'/*')
        im_files = im_files+name_images_per_class
        size_classes.append(len(name_images_per_class))
    labels = np.zeros((len(im_files),len(classes)))
    ant = 0
    for id_i,i in enumerate(size_classes):
        labels[ant:ant+i,id_i] = 1
        ant = i
    collection = imread_collection(im_files)
    data = []
    for id_i,i in enumerate(collection):
        data.append((i.reshape(1,-1)))
    return np.asarray(data), np.asarray(labels)

In [ ]:
path = 'database/'

data, labels = read_images(path)

batch_size = 5

epochs = 31

percent = 0.5

data_size = len(data)
idx = np.arange(data_size)
random.shuffle(idx) 
data = data[idx]
labels = labels[idx]

train = (data[0:np.int(data_size*percent),:,:],labels[0:np.int(data_size*percent),:])

test = (data[np.int(data_size*(1-percent)):,:,:],labels[np.int(data_size*(1-percent)):,:])

train_size = len(train[0])

sess = tf.InteractiveSession()

tf.initialize_all_variables().run()

for n in range(epochs):
    for i in range(int(np.ceil(train_size/batch_size))):
        if (i*batch_size+batch_size <= train_size):
            batch = (train[0][i*batch_size:i*batch_size+batch_size],
                     train[1][i*batch_size:i*batch_size+batch_size])
        else:
            batch = (train[0][i*batch_size:],
                     train[1][i*batch_size:])
            
        train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
        
    if(n%5 == 0):
        train_accuracy = accuracy.eval(feed_dict={x:batch[0], y_: batch[1], keep_prob: 1.0})
        print("Epoca %d, acuracia do treinamento = %g"%(n, train_accuracy))



## Classification

In [ ]:
acuracia = accuracy.eval(feed_dict={x: test[0][:], y_: test[1][:], keep_prob: 1.0})
print("Acuracia = ", acuracia)